In [2]:
# Test this with Llama 3.2 3B and 1B models

import os
from dotenv import load_dotenv
import requests
import os
import json
import chromadb
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Define the endpoint and model
url = "http://localhost:1234/v1/completions"
model = "hugging-quants/Llama-3.2-1B-Instruct-Q8_0-GGUF"

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

/home/darrell/Documents/GitHub/ToolHub/venv_test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Directly Passing Tools in Prompt

In [31]:

query = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Questions: Can you tell me the weather for Cape Coral? 

Here is a list of functions in JSON format that you can invoke:
[
    {
        "name": "turn_on_light",
        "description": "Send an HTTP request to turn on a smart lightbulb in a specific room.",
        "parameters": {
            "type": "dict",
            "required": [
                "room"
            ],
            "properties": {
                "room": {
                    "type": "string",
                    "description": "The room where the light is located. Must be one of: living room, kitchen, office, bedroom, or garage.",
                    "enum": ["livingroom", "kitchen", "office", "bedroom", "garage"]
                }
            }
        }
    },

    {
        "name": "fetch_weather",
        "description": "Fetch the current weather for a given location.",
        "parameters": {
            "type": "dict",
            "required": [
                "location"
            ],
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the city or coordinates (latitude,longitude) to fetch the weather for."
                },
                "units": {
                    "type": "string",
                    "description": "The unit system for temperature (metric for Celsius, imperial for Fahrenheit).",
                    "enum": ["metric", "imperial"],
                    "default": "metric"
                }
            }
        }
    },
    {
        "name": "set_alarm",
        "description": "Set an alarm for a specific time.",
        "parameters": {
            "type": "dict",
            "required": [
                "time"
            ],
            "properties": {
                "time": {
                    "type": "string",
                    "description": "The time to set the alarm for, in HH:MM 24-hour format."
                },
                "label": {
                    "type": "string",
                    "description": "An optional label for the alarm.",
                    "default": "Alarm"
                }
            }
        }
    },
    
]

Should you decide to return the function call, Put it in the format of func(params_name=params_value, params_name2=params_value2...)
NO other text MUST be included.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
# Define the payload for the API request
data = {
    "model": model,
    "prompt": query,
}

# Make the API request
response = requests.post(url, json=data)

# Check if the response was successful
if response.status_code == 200:
    result = response.json()
    print(result['choices'][0]['text'].strip())  # Extract and print the response text
else:
    print(f"Error: {response.status_code}, {response.text}")


"fetch_weather(location='Cape Coral', units='metric')"


## Using Most Similar Tool from Vector Database

In [3]:
# Initialize ChromaDB (Persistent)
chroma_client = chromadb.PersistentClient(path="../database/chroma")
collection = chroma_client.get_or_create_collection(name="tool_json_documents")

# Load local embedding models
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:


# Query Example
query = "Can you turn on the light in the garage?"
query_embedding = embedding_model.encode(query, convert_to_numpy=True).tolist()

results = collection.query(query_embeddings=[query_embedding], n_results=1)

json_tool = {}
# Print the top result
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    # print(f"🔍 Retrieved Document from {meta['filename']}:\n", "\n")
    json_tool = json.loads(doc)
    json_tool = json_tool['tool']




json_tool = {}
# Print the top result
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    # print(f"🔍 Retrieved Document from {meta['filename']}:\n", "\n")
    json_tool = json.loads(doc)  # Convert string to JSON
    json_tool = json_tool['tool']  # Extract the relevant 'tool' part

# Format the JSON with indentation (using tabs)
formatted_json = json.dumps(json_tool, indent=4)  # You can use 4 spaces, or a tab character as you prefer

# Construct the updated query
updated_query = f"""
<|begin_of_text|>

<|start_header_id|>system<|end_header_id|>
You are an expert in composing function calls. You are given a question and a set of possible functions. 
Your task is to determine which function(s) should be invoked to fulfill the request.  

- If none of the functions are applicable, state that explicitly.  
- If required parameters are missing, point that out.  
- You MUST return function calls **ONLY** in the following format:  
  **[func_name1(param1=value1, param2=value2...), func_name2(param=value)]**  
- DO NOT return JSON, explanations, or any other text.  

Here is a list of available functions in JSON format for reference:  
{formatted_json}

Your response MUST contain **only** the function call(s) in the exact format specified.  
**DO NOT** include JSON, extra text, or explanations.  

<|eot_id|><|start_header_id|>end_header_id|>

<|start_header_id|>user<|end_header_id|>
Question: {query}
"""


# Define the payload for the API request
data = {
    "model": model,
    "prompt": updated_query,
}

# Make the API request
response = requests.post(url, json=data)

# Check if the response was successful
if response.status_code == 200:
    result = response.json()
    print(result['choices'][0]['text'].strip())  # Extract and print the response text
else:
    print(f"Error: {response.status_code}, {response.text}")


[turn_on_light(room=garage)]
